In [2]:
from transformers import pipeline
import torch
import pandas as pd
import numpy as np
import polars as pl

```@inproceedings{camacho-collados-etal-2022-tweetnlp,
    title = "{T}weet{NLP}: Cutting-Edge Natural Language Processing for Social Media",
    author = "Camacho-collados, Jose  and
      Rezaee, Kiamehr  and
      Riahi, Talayeh  and
      Ushio, Asahi  and
      Loureiro, Daniel  and
      Antypas, Dimosthenis  and
      Boisson, Joanne  and
      Espinosa Anke, Luis  and
      Liu, Fangyu  and
      Mart{\'\i}nez C{\'a}mara, Eugenio" and others,
    booktitle = "Proceedings of the 2022 Conference on Empirical Methods in Natural Language Processing: System Demonstrations",
    month = dec,
    year = "2022",
    address = "Abu Dhabi, UAE",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2022.emnlp-demos.5",
    pages = "38--49"
}
```

- F1 (micro): 0.6931034482758621
- F1 (macro): 0.692628774202147
- Accuracy: 0.6931034482758621

In [3]:
filtered_df = pd.read_parquet(
    "filtered_sport_category_with_sentiment_column_metadata.parquet")

In [4]:
filtered_df

,categories,channel_id,crawl_date,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,sentiment,sentiment_score,sentiment_title,sentiment_score_title,sentiment_description,sentiment_score_description
0,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:39:57.427254,35.0,JOeSxtcNdHQ,8620,1673.0,"catfishing,fishing,fishing challenge,catfish,c...",Catching 100 lbs of Catfish 🔴Live,2019-10-01 00:00:00,48737.0,neutral,0.801140,neutral,0.880023,neutral,0.718099
1,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:39:58.108323,15.0,EPMLTw2zINw,355,1297.0,,big cat,2019-10-01 00:00:00,19999.0,positive,0.589074,positive,0.589074,negative,0.347396
2,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:39:58.773085,78.0,Y1_pK68iSYQ,603,3305.0,"Catfishing,how to catch catfish,fishing,classy...",Classy Catfishing - How to Catch Catfish the P...,2019-09-28 00:00:00,58518.0,neutral,0.592033,neutral,0.508985,neutral,0.580420
3,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:39:59.465346,70.0,jF8TSo3ZfTc,1426,1889.0,"Fishing,Fishing uk,Angling,Sea angling,sea ang...",2 Day Saltwater Fishing Catch & Cook - UK Sea ...,2019-09-21 00:00:00,71998.0,negative,0.866445,neutral,0.987322,negative,0.819963
4,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:40:00.188768,73.0,Gp00dNaVouo,990,2699.0,"Fishing,catfish,wels catfish,how to catch catf...",How to Catch Wels Catfish - Fishing for Catfis...,2019-09-14 00:00:00,101924.0,neutral,0.874077,neutral,0.949094,neutral,0.845538
5,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:40:00.909515,126.0,luVmvoNN6Yw,869,2823.0,"Fishing,Carp fishing,fishing for carp,How to c...",How to Catch Carp in a Small Pond - Carp Fishi...,2019-09-07 00:00:00,117106.0,neutral,0.862109,neutral,0.977251,neutral,0.863332
6,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:40:01.612961,54.0,3qQgiZZ66vw,528,3122.0,"fishing,fishing rod,palm frond,fishing challen...",Making a Fishing Pole from Palm Fronds - Homem...,2019-08-31 00:00:00,52983.0,neutral,0.608147,neutral,0.697675,neutral,0.725342
7,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:40:02.318070,57.0,OX7f0Bfp4s0,710,2165.0,"fishing,taxidermy,fish mount,mounting fish,gyo...",How to Mount a Fish Cheap & Easy - Gyotaku (Ja...,2019-08-24 00:00:00,47952.0,positive,0.628846,neutral,0.962192,positive,0.766546
8,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:40:03.004935,166.0,jc8yJCssT_0,803,3943.0,"Catch and Cook,Catch clean cook,Catch and cook...","Catch and Cook Striped Bass - Catching, cleani...",2019-08-17 00:00:00,185625.0,negative,0.960168,neutral,0.901880,negative,0.961665
9,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,2019-11-03 16:40:03.721402,89.0,2zxR9X8ZG9w,845,2787.0,"Fishing,carp,how to catch carp,carp bait,bait,...",How to Catch Carp with My Favorite Carp Baits ...,2019-08-10 00:00:00,147103.0,neutral,0.783741,neutral,0.878422,neutral,0.857205
